# Convex Regression

Convex regression experiments on synthetic problems.

### Important parameters:
    - global_random_seed: the first initialization seed of the random number generator
    - parallel_workers: the maximum number of parallel jobs (consider available RAM for choosing this too)
    - domain_dims: domain dimensions (each of them defines a separate experiment)
    - nsamples: sample sizes (each of them defines a separate experiment)
    - nruns: number of runs for each experiment (statistics like mean, std, etc... are evaluated over the runs)
    - estimators: estimators to be evaluated (uncomment the ones you want below)
    - ntestsamples: number of test samples to use for the evaluation
    
Specify all the estimators to be used for all experiments in the *Estimators* section.<br>
Specify one regression problem used for all experiments in the *Problem setting* section. 

In [ ]:
experiment_id = '_MISSING_ID'  # Name your experiment here!

In [ ]:
%autosave 120
%pylab inline

In [ ]:
import os
import sys
import time
import logging

project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
print('project_path: {}'.format(project_path))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from joblib import Parallel, delayed, Memory
from collections import OrderedDict
from IPython.display import display

from common.util import set_random_seed

In [ ]:
logging.basicConfig(
    handlers=(
        # logging.FileHandler('.../file.log'),
        logging.StreamHandler(sys.stdout),
    ),
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
    format='%(asctime)s|%(levelname)s|%(message)s',
)

def info(*args):
    logging.info('PID:{}|'.format(os.getpid()) + args[0] + '\n', *args[1:])

In [ ]:
nruns = 10  # number of experiment runs
ntestsamples = int(1e6)  # number of test samples to generate

parallel_nworkers = 1  # maximum number of parallel workers (make sure you have enough RAM too)
parallel_backend = 'multiprocessing'

In [ ]:
seed_limit = 1e6
global_random_seed = 100 + int(np.round((time.time() % 1) * seed_limit))
set_random_seed(global_random_seed)
setup_random_seed = np.random.randint(seed_limit)
data_random_seed = np.random.randint(seed_limit)
training_random_seed = np.random.randint(seed_limit)
testing_random_seed = np.random.randint(seed_limit)
info('random seeds, global:{}, setup:{}, data:{}, training:{}, testing:{}'.format(
    global_random_seed, setup_random_seed, data_random_seed,
    training_random_seed, testing_random_seed,
))

## Estimators

In [ ]:
set_random_seed(setup_random_seed)
estimators = OrderedDict()

def get_estimator(estimator_name):
    return estimators[estimator_name]

In [ ]:
# Ordinary Least-Squares estimator
from common.ols import OLSEstimator
estimators['OLS'] = OLSEstimator()

In [ ]:
# # LSPA
# from y2009_lspa.lspa import LSPAEstimator
# estimators['LSPA'] = LSPAEstimator(train_args={'ncenters': 'n**(d/(d+4))', 'nrestarts': 'd', 'nfinalsteps': 'n'})

In [ ]:
# CNLS
from y2004_cnls.cnls import CNLSEstimator
estimators['CNLS_star'] = CNLSEstimator(train_args={'use_L': True})
estimators['CNLS_ln'] = CNLSEstimator(train_args={'use_L': True, 'ln_L': True})

In [ ]:
# # Convex Adaptive Partitioning (CAP)
# from y2013_cap.cap import CAPEstimator
# estimators['CAP'] = CAPEstimator()
# # estimators['FastCAP'] = CAPEstimator(train_args={'nranddirs': 5})

In [ ]:
# # PCNLS with random Voronoi partition
# from y2015_pcnls.pcnls_voronoi import PCNLSVoronoiEstimator
# estimators['PCNLS-Voronoi'] = PCNLSVoronoiEstimator()

In [ ]:
# # Adaptive Max-Affine Partitioning (AMAP)
# from y2016_amap.amap import AMAPEstimator
# estimators['AMAP'] = AMAPEstimator()

In [ ]:
# APCNLS
from y2022_apcnls.apcnls import APCNLSEstimator
estimators['APCNLS_star'] = APCNLSEstimator(train_args={'use_L': True})
estimators['APCNLS_ln'] = APCNLSEstimator(train_args={'use_L': True, 'ln_L': True})

## Caching

In [ ]:
is_caching_enabled = (global_random_seed < 100)  # caching is pointless without manual random seed setting
if is_caching_enabled:
    cache_dir = os.path.join(project_path, '_result_cache', experiment_id)
    persister_dict = {}
    for estimator_name in estimators.keys():
        estimator_cache_dir = os.path.join(cache_dir, estimator_name)
        os.makedirs(estimator_cache_dir, exist_ok=True)
        persister_dict[estimator_name] = Memory(estimator_cache_dir, verbose=2)

def cached_func(func, estimator_name):
    if is_caching_enabled:
        return persister_dict[estimator_name].cache(func)
    return func

## Problem setting

In [ ]:
domain_dims = (3,)  # domain dimensions
nsamples = (100, 250)  # number of samples
L = np.inf  # Lipschitz limit (can be set as a function to measure L on the union of the training and test sets)
L_scaler = 1.0  # multiplying L (makes sense when L is measured on the data)

In [ ]:
def loss(yhat, y):  # L2-error
    return np.mean(np.square(yhat - y))

#### Regression function

In [ ]:
# # Linear regression function
# def fstar(X):
#     return np.sum(X, axis=1)

# L = 1.0

In [ ]:
# # Symmetric L1-norm regression function
# def fstar(X):
#     return np.sum(np.abs(X), axis=1)

# def L_func(X):
#     return max(np.linalg.norm(np.sign(X), ord=2, axis=1))

# L = L_func

In [ ]:
# # Truncated L1-norm regression function
# def fstar(X):
#     return np.sum(np.abs(np.maximum(X, 0.0)), axis=1)

# def L_func(X):
#     return max(np.linalg.norm(np.sign(np.maximum(X, 0.0)), ord=2, axis=1))

# L = L_func

In [ ]:
# Symmetric quadratic regression function
def fstar(X):
    return 0.5 * np.sum(np.square(X), axis=1)

def L_func(X):
    return max(np.linalg.norm(X, ord=2, axis=1))

L = L_func

In [ ]:
# # Truncated quadratic regression function
# def fstar(X):
#     return 0.5 * np.sum(np.square(np.maximum(X, 0.0)), axis=1)

# def L_func(X):
#     return max(np.linalg.norm(np.maximum(X, 0.0), ord=2, axis=1))

# L = L_func

#### Covariate distribution

In [ ]:
# full-dimensional Gaussian covariate
covariate_std = 1.0

def sample_X(n, d):
    return np.random.randn(n, d) * covariate_std

In [ ]:
# # full-dimensional Uniform covariate
# covariate_min = -1.0
# covariate_max = 1.0

# def sample_X(n, d):
#     return np.random.rand(n, d) * (covariate_max - covariate_min) + covariate_min

In [ ]:
# # Uniform random variable linearly embedded into a larger space with Gaussian measurement noise
# low_d = 3
# covariate_min = -3.0
# covariate_max = 3.0
# measurement_noise_std = 0.1

# def sample_X(n, d):
#     X = np.random.randn(n, d) * measurement_noise_std
#     X[:, :low_d] = np.random.rand(n, low_d) * (covariate_max - covariate_min) + covariate_min
#     return X

In [ ]:
# # Uniform random variable with polynomial expansion and Gaussian measurement noise
# covariate_min = -1.0
# covariate_max = 1.0
# measurement_noise_std = 0.1

# def sample_X(n, d):
#     X = np.random.randn(n, d) * measurement_noise_std
#     Z = np.random.rand(n) * (covariate_max - covariate_min) + covariate_min
#     for power in range(d):
#         X[:, power] += Z**power
#     return X

#### Observation noise distribution

In [ ]:
# Gaussian observation noise
observation_noise_std = 0.3

def sample_noise(n):
    return np.random.randn(n) * observation_noise_std

In [ ]:
# # Rademacher observation noise

# def sample_noise(n):
#     return 2.0 * (np.random.randint(0, 2, n) - 0.5)

## Training

In [ ]:
def get_random_seed_offset(d, n, run):
    return d * n + run


def get_data(d, n, run, data_random_seed):
    set_random_seed(data_random_seed + get_random_seed_offset(d, n, run))

    X = sample_X(n, d)
    y_true = fstar(X)
    y = y_true + sample_noise(n)

    X_test = sample_X(ntestsamples, d)
    y_test = fstar(X_test)

    return X, y, y_true, X_test, y_test


def run_experiment(d, n, L, estimator_name, run, data_random_seed, training_random_seed):
        X, y, y_true, X_test, y_test = get_data(d, n, run, data_random_seed)
        L_true = max(L(X), L(X_test)) if callable(L) else L
        Lscaler = eval(L_scaler) if isinstance(L_scaler, str) else L_scaler
        L_est = (L_true * Lscaler) if np.isfinite(L_true) else np.inf

        X_norms = np.linalg.norm(X, axis=1)
        X_test_norms = np.linalg.norm(X_test, axis=1)
        info(('\nExperiment, d: {}, n: {}, estimator: {}, L_true: {:.1f}, run: {},\n'
              'train data, minX: {:.2f}, maxX: {:.2f}, minXnorm: {:.4f}, maxXnorm: {:.2f},\n'
              '            miny: {:.2f}, meany: {:.4f}, stdy: {:.4f}, maxy: {:.2f},\n'
              ' test data, minX: {:.2f}, maxX: {:.2f}, minXnorm: {:.4f}, maxXnorm: {:.2f},\n'
              '            miny: {:.2f}, meany: {:.4f}, stdy: {:.4f}, maxy: {:.2f},\n').format(
            d, n, estimator_name, L_true, run,
            np.min(X), np.max(X), np.min(X_norms), np.max(X_norms),
            np.min(y), np.mean(y), np.std(y), np.max(y),
            np.min(X_test), np.max(X_test), np.min(X_test_norms), np.max(X_test_norms),
            np.min(y_test), np.mean(y_test), np.std(y_test), np.max(y_test),
        ))
        set_random_seed(training_random_seed + get_random_seed_offset(d, n, run))
        result = OrderedDict()
        estimator = get_estimator(estimator_name)

        train_args = OrderedDict()
        if np.isfinite(L_est):
            train_args['L'] = L_est
        result['L_est'] = L_est
        result['L_true'] = L_true

        real_time, cpu_time = time.time(), time.clock()
        model = estimator.train(X, y, **train_args)
        result['model'] = model
        result['nweights'] = model.weights.shape[0]
        result['max_weight_norm'] = max(np.linalg.norm(model.weights, axis=1))
        yhat = estimator.predict(model, X)
        result['train_risk'] = loss(yhat, y)
        result['train_err'] = loss(yhat, y_true)
        result['train_diff_mean'] = np.mean(yhat - y)
        result['train_diff_median'] = np.median(yhat - y)
        result['train_cpu_time'] = time.clock() - cpu_time
        result['train_real_time'] = time.time() - real_time

        real_time, cpu_time = time.time(), time.clock()
        yhat_test = estimator.predict(model, X_test)
        result['test_err'] = loss(yhat_test, y_test)
        result['test_cpu_time'] = time.clock() - cpu_time
        result['test_real_time'] = time.time() - real_time

        info(('\nResult, d: {}, n: {}, estimator: {}, L_est: {:.1f}, run: {},\n'
              ' train, err: {:.4f}, risk: {:.4f}, real_time: {}s,\n'
              '  test, err: {:.4f}, real_time: {}s').format(
            d, n, estimator_name, L_est, run,
            result['train_err'], result['train_risk'], int(np.ceil(result['train_real_time'])),
            result['test_err'], int(np.ceil(result['test_real_time'])),
        ))
        return ((d, n, estimator_name, run), result)

In [ ]:
results = []
delayed_funcs = []
for d in domain_dims:
    for n in nsamples:
        for estimator_name in estimators.keys():
            for run in range(nruns):
                delayed_funcs.append(delayed(cached_func(run_experiment, estimator_name))(
                    d, n, L, estimator_name, run,
                    data_random_seed, training_random_seed,
                ))
results = OrderedDict(sorted(Parallel(n_jobs=parallel_nworkers, backend=parallel_backend)(delayed_funcs)))
info('All results have been calculated.')

## Evaluation

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

def collect_stat_keys_and_values(results, estimator_name):
    stat_keys = set()
    stat_values = OrderedDict()
    for k, r in results.items():
        if k[-2] != estimator_name:
            continue
        stat_values.setdefault(k[:-2], []).append(r)
        for sk in r.keys():
            stat_keys.add(sk)
    return stat_keys, stat_values

In [ ]:
# Printing common statistics.

skipped_stats = ('model',)
stat_funcs = OrderedDict((
    ('mean', np.mean),
    ('std', np.std),
    ('min', np.min),
    ('median', np.median),
    ('max', np.max),
))

ds = set()
stats = OrderedDict()
for estimator_name in estimators.keys():
    stat_keys, stat_values = collect_stat_keys_and_values(results, estimator_name)
    stat = {}
    for (d, n), s in stat_values.items():
        ds.add(d)
        ss = OrderedDict()
        for sk in stat_keys:
            if sk in skipped_stats:
                continue
            for sf_name, sf in stat_funcs.items():
                ss[sk + '__' + sf_name] = sf([v[sk] for v in s])
        stat[(d, n)] = ss
    stat = pd.DataFrame(stat)
    stat.columns.names = ('d', 'n')
    print('\nestimator: {}'.format(estimator_name))
    stats[estimator_name] = stat
    display(stat)

In [ ]:
# Plotting common statistics.

skipped_estimators = []  # ['OLS']

for d in ds:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
    for estimator_name, stat in stats.items():
        if estimator_name in skipped_estimators:
            continue

        stat = stat.T
        stat = stat[stat.index.get_level_values(0) == d]
        if not stat.empty:
            ax1.set_title('d: {}, nruns: {}'.format(d, nruns))
            ax1.set_xlabel('n')
            ax1.set_ylabel('test error')
            ax1.errorbar(
                x=stat.index.get_level_values(1),
                y=stat['test_err__mean'],
                yerr=stat['test_err__std'],
                label=estimator_name,
            )
            ax1.legend(loc='upper right')

            ax2.set_title('d: {}, nruns: {}'.format(d, nruns))
            ax2.set_xlabel('n')
            ax2.set_ylabel('training risk')
            ax2.errorbar(
                x=stat.index.get_level_values(1),
                y=stat['train_risk__mean'],
                yerr=stat['train_risk__std'],
                label=estimator_name,
            )
            ax2.legend(loc='upper right')

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
    for estimator_name, stat in stats.items():
        if estimator_name in skipped_estimators:
            continue

        stat = stat.T
        stat = stat[stat.index.get_level_values(0) == d]
        if not stat.empty:
            ax1.set_title('d: {}, nruns: {}'.format(d, nruns))
            ax1.set_xlabel('n')
            ax1.set_ylabel('number of weight vectors')
            ax1.errorbar(
                x=stat.index.get_level_values(1),
                y=stat['nweights__mean'],
                yerr=stat['nweights__std'],
                label=estimator_name,
            )
            ax1.legend(loc='upper right')

            ax2.set_title('d: {}, nruns: {}'.format(d, nruns))
            ax2.set_xlabel('n')
            ax2.set_ylabel('training time (sec)')
            ax2.errorbar(
                x=stat.index.get_level_values(1),
                y=stat['train_cpu_time__mean'],
                yerr=stat['train_cpu_time__std'],
                label=estimator_name,
            )
            ax2.legend(loc='upper left')

In [ ]:
# Printing estimator model specific statistics.

estimator_names = []
model_fields = []

stats = OrderedDict()
for estimator_name in estimators.keys():
    if estimator_name not in estimator_names:
        continue
    stat_keys, stat_values = collect_stat_keys_and_values(results, estimator_name)
    stat = {}
    for (d, n), s in stat_values.items():
        ss = OrderedDict()
        for sk in stat_keys:
            if sk != 'model':
                continue
            for field in model_fields:
                for sf_name, sf in stat_funcs.items():
                    vals = [v for v in [getattr(v[sk], field) for v in s] if v is not None]
                    ss[field + '__' + sf_name] = None if len(vals) == 0 else sf(vals)
        stat[(d, n)] = ss
    stat = pd.DataFrame(stat)
    stat.columns.names = ('d', 'n')
    print('\nestimator: {}'.format(estimator_name))
    stats[estimator_name] = stat
    display(stat)